# Summarizing the raw files

The 218 inverter data files from the SunPower data set contain data of unknown length and quality. Some files have data from 2014 through 2017, while other files represent systems built recently and have less than a year's worth of data. In addition, some sites have experienced outages (either PV or DAS) and are missing data. This notebook inspects summary data for all the files.

In [1]:
from core.preprocessing import summarize_files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, DateFormatter
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid', palette='colorblind', context='talk')
from simplekml import Kml

In [ ]:
path_to_files = '/Users/bennetmeyers/Documents/CS229/Project/data_dump/'

In [ ]:
%time summary = summarize_files(path_to_files, suffix='pkl')

218 files to process


In [ ]:
len(summary)

In [ ]:
summary.head()

In [ ]:
_ = plt.hist(summary['num_vals'], bins=30)

In [ ]:
fig, axes = plt.subplots(nrows=2, figsize=(12,8), sharex=True)
axes[0].fill_between(summary.index, date2num(list(summary['t_start'])), date2num(list(summary['t_end'])))
xfmt = DateFormatter('%Y-%m-%d')
axes[0].yaxis.set_major_formatter(xfmt)
axes[1].plot(summary.index, summary['num_vals'])
plt.xlabel('Inverter Index Number')
axes[0].set_ylabel('Start/End Dates')
axes[1].set_ylabel('Data Count')

In [ ]:
summary.plot(y='ac_max')

The first 10 or so files appear to have AC power in Watts, while all the rest are in kW.

In [ ]:
indexer = summary['ac_max'] > 2000
summary.loc[indexer, ['ac_max']] = summary.loc[indexer, ['ac_max']] / 1000.
summary.loc[indexer, ['ac_min']] = summary.loc[indexer, ['ac_max']] / 1000.
summary.loc[indexer, ['ac_avg']] = summary.loc[indexer, ['ac_max']] / 1000.
summary.loc[indexer, ['ac_stdev']] = summary.loc[indexer, ['ac_max']] / 1000.

In [ ]:
summary.plot(y='ac_max')
plt.ylim(0, 40)

In [ ]:
summary.plot(y='ac_avg')
plt.ylim(0, 40)

In [ ]:
plt.plot(summary['ac_max'] / summary['ac_avg'])

In [ ]:
selection = np.alltrue([
    summary['t_start'] <= '2015-07-15',
    summary['t_end'] >= '2017-07-15',
    summary['num_vals'] >= 100000,
    summary['ac_max'] / summary['ac_avg'] <= 10
], axis=0)

In [ ]:
num_inverters = np.sum(selection)
num_sites = len(set(summary[selection]['site ID']))
print 'This selection yields {} sites and {} inverters'.format(num_sites, num_inverters)

In [ ]:
selected_sites = set(summary[selection]['site ID'])

Load the external metadata file containing the lat/long coordinates.

In [ ]:
metadata_fp = '/Users/bennetmeyers/Documents/Vader/data/metadata_files/deviceMetaData_20171108_LatLon.csv'
site_metadata = pd.read_csv(metadata_fp)

In [ ]:
site_metadata.set_index('serialNumber', inplace=True)

Plot the spatial distribution

In [ ]:
sns.set_style('white')
coords = site_metadata.loc[selected_sites, ['lat', 'lon']]
plt.scatter(coords['lon'], coords['lat'])
ax = plt.gca()
ax.ticklabel_format(style='plain', axis='x')
plt.title('Spatial Distribution of Sites')
plt.ylabel('latitude')
_ = plt.xlabel('longitude')

Write a KML file mapping the locations of the selected sites.

In [ ]:
my_kml = Kml()
for row in site_metadata.loc[selected_sites].iterrows():
    str1 = (row[0]+'<br/>'+'# of inverters: {}'.format(row[1]['inverters'])+
            '<br/>'+'type: '+row[1]['addressType'])
    pnt = my_kml.newpoint(description=str1, coords=[(row[1]['lon'], row[1]['lat'])])
    if row[1]['addressType'] == 'Residential':
        pnt.style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/paddle/ylw-blank.png'
    elif row[1]['addressType'] == 'Commercial':
        pnt.style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/paddle/blu-blank.png'
    else:
        pnt.style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/paddle/wht-blank.png'

In [ ]:
my_kml.save("selected_sites.kml")

In [ ]:
np.savetxt('selected_sites.txt', np.asarray(list(selected_sites)), fmt='%s')